In [1]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [2]:
os.environ['_JAVA_OPTIONS'] = f'-Djava.io.tmpdir={localfs_path}'

import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail'),
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '10G'}, # I don't know what should be here
    default_reference='GRCh38'
) 

Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/3688449/
Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/3688449/


23/06/21 21:24:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://ac0040:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.113-cf32652c5077
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230621-2124-0.2.113-cf32652c5077.log


In [3]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [4]:
poles = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/synthetic-controls-no-zeros.mt')

In [43]:
par1 = [10001,2781479]
par2 = [155701383,156030895]

nonpars = [[1,10000],[2781480,155701382]]

polish_samples = []

for i in range(1,1001):
    polish_samples.append(str(i)+'_polish')

poles_females = polish_samples[0:150] 
poles_males = polish_samples[150:]

path_names = ['poles_zeros', 'poles']

poles = poles.annotate_cols(
        sex = hl.if_else(
            hl.array(poles_females).contains(poles.s),
            'F',
            'M'
        )
    )
    
poles = poles.annotate_entries(
        GT = hl.if_else(
            poles.locus.contig == 'chrX',
            hl.if_else(
                poles['sex'] == 'F',
                poles.GT,
                hl.if_else(
                    poles.locus.position > nonpars[1][1],
                    poles.GT,
                    hl.if_else(
                        poles.locus.position < nonpars[0][1],
                        hl.if_else(
                            poles.locus.position > nonpars[0][0],
                            poles.GT,
                            hl.call(poles.GT[0], poles.GT[0])),
                        hl.call(poles.GT[0], poles.GT[0])))),
            poles.GT
        )
    )
    
poles = poles.checkpoint(localfs_path+'poles_sex.mt')  

FatalError: HailException: path '/localfs/3688449/poles_sex.mt' is both an input and output source in a single query. Write, export, or checkpoint to a different path!

Java stack trace:
is.hail.utils.HailException: path '/localfs/3688449/poles_sex.mt' is both an input and output source in a single query. Write, export, or checkpoint to a different path!
	at is.hail.utils.ErrorHandling.fatal(ErrorHandling.scala:17)
	at is.hail.utils.ErrorHandling.fatal$(ErrorHandling.scala:17)
	at is.hail.utils.package$.fatal(package.scala:78)
	at is.hail.expr.Validate$.fileReadWriteError(Validate.scala:14)
	at is.hail.expr.Validate$.$anonfun$validate$2(Validate.scala:24)
	at is.hail.expr.Validate$.$anonfun$validate$2$adapted(Validate.scala:22)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:38)
	at is.hail.expr.Validate$.val

In [ ]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/gts_and_s.mt')

mt = mt.drop(
    mt.group, mt.sample_qc
)

mt = mt.select_entries(
    AD = mt.LAD,
    DP = mt.DP,
    GQ = mt.GQ,
    GT = mt.GT,
    MIN_DP = mt.MIN_DP,
    PGT = mt.LPGT,
    PID = mt.PID,
    PL = mt.LPL,
    PS = mt.PS,
    RGQ = mt.RGQ,
    SB = mt.SB
)

mt = mt.annotate_cols(sex = hl.missing(hl.tstr))


samples_to_exclude = ['S_7288', 'S_7289', 'S_7290', 'WGS_6827', 'WGS_6835']
#these are low quality samples to exclude

mt = mt.filter_cols(
    hl.literal(samples_to_exclude).contains(mt.s),
    keep = False
)

pl = hl.read_matrix_table(localfs_path+'poles_sex.mt')  
pl = pl.drop(pl.col_idx)
pl = mt.union_cols(pl)
pl.write(localfs_path+'pw_s_gts_union_2.mt')

In [ ]:
mt = hl.read_matrix_table(localfs_path+'pw_s_gts_union_2.mt')

In [ ]:
mt.count()

In [ ]:
to_keep = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/s_vs_gts.mt'
)

In [ ]:
to_keep.aggregate_cols(hl.agg.counter(to_keep.group))

In [ ]:
pheno = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/pheno/GTS-coded-corrected-june-2021.csv',
    impute=True,
    delimiter=',',
    quote="\""
    )

pheno = pheno.key_by(pheno.ID)

mt = mt.annotate_cols(phenotypes = pheno[mt.s])

In [ ]:
mt = mt.annotate_cols(
    group = hl.if_else(
        mt.s.contains('B'),
        'local_controls',
        hl.if_else(
            mt.s.contains('polish'),
            'polish_controls',
            'GTS'
        )
    )
) 

In [ ]:
mt = mt.filter_cols(
    (
        mt.group == 'polish_controls') | (
        mt.group == 'local_controls') | (
        mt.phenotypes.family != '.') | (
        hl.is_defined(to_keep.cols()[mt.col_key])
    )
)

In [ ]:
mt = mt.checkpoint(localfs_path+'pre_pca_gts.mt')

In [ ]:
bed = hl.import_bed(
    '/net/pr2/projects/plgrid/plggneuromol/matzieb/projects/imdik-zekanowski-sportwgs/data/prs-data/all-alt-scaffold-placement-GRCh38.p14.bed'
)

In [ ]:
mt = mt.filter_rows(
    hl.is_snp(mt.alleles[0], mt.alleles[1]) & 
    mt.was_split == False &
    hl.is_missing(bed[mt.locus])
)  

### RUN PCA for everyone

In [37]:
#mt = mt.checkpoint(localfs_path+'pre_pca_gts_filtered.mt')
mt = hl.read_matrix_table(localfs_path+'pre_pca_gts_filtered.mt')

In [38]:
mt.count()

(3042073, 1280)

In [39]:
def run_pca_no_filter(mtx, mtx_path, suffix):
    
    for_pca = mtx.sample_rows(0.2)

    for_pca = for_pca.checkpoint(localfs_path+'subset_'+suffix+mtx_path)
    for_pca = hl.read_matrix_table(localfs_path+'subset_'+suffix+mtx_path)
    eigenvalues, pcs, _ = hl.hwe_normalized_pca(for_pca.GT, k=20)
    
    mtx = mtx.annotate_cols(scores_no_filter = pcs[mtx.s].scores)
    mtx = mtx.checkpoint(localfs_path+'after_pca_no_filters_'+suffix+mtx_path)
    
    return(mtx)

In [40]:
mt = run_pca_no_filter(mt, 'all_samples.mt', '3_')

2023-06-22 14:24:03.946 Hail: INFO: wrote matrix table with 606866 rows and 1280 columns in 4389 partitions to /localfs/3688449/subset_3_all_samples.mt
    Total size: 6.12 GiB
    * Rows/entries: 6.12 GiB
    * Columns: 8.46 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  897 rows (9.44 MiB)
2023-06-22 14:24:44.293 Hail: INFO: hwe_normalize: found 598852 variants after filtering out monomorphic sites.
2023-06-22 14:25:01.509 Hail: INFO: pca: running PCA with 20 components... 4389]
2023-06-22 14:28:25.582 Hail: INFO: wrote table with 0 rows in 0 partitions to /localfs/3688449/tmp_hail/persist_tableitCUHond33
    Total size: 208.84 KiB
    * Rows: 0.00 B
    * Globals: 208.84 KiB
    * Smallest partition: N/A
    * Largest partition:  N/A
2023-06-22 14:43:21.125 Hail: INFO: wrote matrix table with 3042073 rows and 1280 columns in 4389 partitions to /localfs/3688449/after_pca_no_filters_3_all_samples.mt


In [41]:
mt = mt.annotate_cols(
    label = hl.if_else(
        mt.group == 'local_controls',
        'Polish elite sportsmen',
        hl.if_else(
            mt.group == 'polish_controls',
            'Polish synthetic controls',
            hl.if_else(
                mt.phenotypes.family == '.',
                'GTS',
                hl.str('GTS family ')+mt.phenotypes.family
            )
        )
    )
)                

In [42]:
p = hl.plot.scatter(mt['scores_no_filter'][0],
                    mt['scores_no_filter'][1],
                    label=mt.label,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/bokeh/models/sources.py:235: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tmp_data = {c: v.values for c, v in _df.iteritems()}
